Natural Language Processing in Tensorflow
---------------------------------------------------------------
1. Sentiment in text
1. **Word Embeddings**
1. Sequence model
1. Sequence models and literature

Word Embeddings
-----------------
- Week1에서 우리는 문장에서 단어들을 어떻게 토큰화하고 그것을 이용해서 어떻게 문장에 대응시키는지 살펴봄
- 토큰화한 단어들이 문장에서 어떻게 배열되느냐도 중요하지만 그 단어가 가지는 가치가 무엇인지 학습하는 것도 중요한 과제임
- Week2에서는 Embedding이라는 과정을 통해서 NN이 어떻게 단어의 의미를 학습할 수 있는지 간단한 예제들을 통해서 살펴보는데 학습 목표를 두고 있음

In [17]:
#!pip install tensorflow==2.5.0

Lab1 실행을 위한 tensorflow 설치 (2.5.0 이상의 버전이 필요)

In [18]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.6.0


- 설치된 TF의 버전 확인
- 더불어, python3 환경인지도 확인 필요

In [19]:
import tensorflow_datasets as tfds
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised=True)


- TensorFlow에서 제공되는 dataset, TFDS(TensorFlow Data Services)를 사용하기 위해서 import
- 본 예제에서는 그 중 영화 리뷰와 관련된 자료인 imdb 자료를 load
- imdb: 많은 양의 데이터와 50,000개의 영화 리뷰를 포함하고 있으며, 이는 긍정적인 리뷰 또는 부정적인 리뷰로 분류되어 있음
- TFDS를 사용하면 train data와 test data를 분류가 이미 되어있으므로 ML과 DL에 사용하기 편리함
- imdb와 info 변수에 위 정보를 저장해두었지만, 본 예제에서는 imdb 변수만 사용할 예정

In [20]:
import numpy as np

# train용 data 25,000개의 샘플과 test용 data 25,000개의 샘플을 나눠서 assign
train_data, test_data = imdb['train'], imdb['test']

# declare lists
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()

# numpy method를 이용해 문장과 라벨들을 텐서의 형태로 추출
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())
  
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())
  
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


- tf.Tensor의 형태로 데이터를 추출하여 저장
- label 중 값 0은 부정적인 리뷰임을 나타내고, 값 1은 긍정적인 리뷰임을 나타냄
- 최종적으로 이 label들은 Numpy Array 형식으로 저장됨

In [21]:
# 문장들을 토큰화하기 위한 hyperparameters
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

# Tokenizer와 pad_sequence을 import
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,maxlen=max_length)


- 문장을 토큰화하는 과정
- Week1에서 수행했던 방법 그대로 따라가며 training_sentences에서 만든 dictionary로 testing_sentence를 sequencing 하는 것이 포인트
- 원활한 학습을 위해서 data의 길이를 같게 해주는 padding 과정을 거침

In [22]:
# 편의를 위해 value와 key의 순서를 reverse
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# decoding 과정
def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[3]))
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


- decode_review(text): decoding하는 과정으로 인덱스의 맞는 단어들을 출력. 이때 ' '.join 함수로 단어 사이에 공백을 두고 출력하게 하고 모르는 단어(OOV)는 '?'를 통해 출력

In [23]:
# Neural Network 정의
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(), #Global average pooling 1D(벡터 전체의 평균을 Flat하게 만듦)
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


- tf.keras.layers.Embedding: 각 단어마다 16차원의 배열을 할당함으로써 단어가 가지는 특성(긍정, 부정)을 16차원의 값을 통해서 표현. 이 과정을 거쳐 학습을 진행하면 자연스럽게 긍정적인 단어는 긍정적인 단어끼리, 부정적인 단어는 부정적인 단어끼리 비슷한 값을 가지게 됨(Cluster 생성)
- tf.keras.layers.Fatten: 2차원 벡터를 input으로 사용할 수 있도록 1차원 벡터로 만드는 역할
- tf.keras.layers.Dense: classification을 위해서 각 relu, sigmoid를 activation fn으로 가지는 layer에 assign
- label 값이 0(부정적)과 1(긍정적)만을 가지기 때문에 binary_crossentropy를 사용
- 추가로 optimizer로 Adam을 사용한 모습을 확인 가능


In [24]:
num_epochs = 10
model.fit(padded, training_labels_final, epochs=num_epochs, validation_data=(testing_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 7s 9ms/step - loss: 0.4868 - accuracy: 0.7496 - val_loss: 0.3477 - val_accuracy: 0.8467
Epoch 2/10
782/782 [==============================] - 7s 8ms/step - loss: 0.2479 - accuracy: 0.9041 - val_loss: 0.3643 - val_accuracy: 0.8414
Epoch 3/10
782/782 [==============================] - 6s 7ms/step - loss: 0.1095 - accuracy: 0.9700 - val_loss: 0.4305 - val_accuracy: 0.8353
Epoch 4/10
782/782 [==============================] - 6s 7ms/step - loss: 0.0338 - accuracy: 0.9947 - val_loss: 0.5099 - val_accuracy: 0.8305
Epoch 5/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0130 - accuracy: 0.9986 - val_loss: 0.5842 - val_accuracy: 0.8269
Epoch 6/10
782/782 [==============================] - 5s 7ms/step - loss: 0.0065 - accuracy: 0.9992 - val_loss: 0.6142 - val_accuracy: 0.8272
Epoch 7/10
782/782 [==============================] - 7s 8ms/step - loss: 0.0031 - accuracy: 0.9996 - val_loss: 0.6796 - val_accuracy: 0.8251
Epoch 

- epoch을 10번으로 설정하고 학습 진행
- Accuracy가 100%지만, Validation이 82.9%에서 멈춘 것으로 보아 Overfitting의 가능성이 있음

In [25]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


- embedding layer의 모습
- 10,000개의 단어를 16차원의 벡터를 통해서 표현한 것을 확인 가능

In [26]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

- Visualization을 위해서 vector data와 meta data를 추출하는 모습
- vecs.tsv: 각 단어에 할당된 16차원 벡터 데이터
- meta.tsv: 각 단어의 토큰화된 모습이 아닌 진짜 모습(ex. amazing, boring... etc.)

In [27]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

- vecs.tsv 파일과 meta.tsv 파일을 직접 다운로드하는 모습
- [projector.tensorflow.org](https://projector.tensorflow.org)에서 데이터를 직접 넣어서 클러스터링 visualization 확인 가능

In [28]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]


- 하나의 예문을 토큰을 이용해서 sequencing한 모습